<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_3G_Ex_Retail51289.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Postgres Exercises <br>
Using the same data used in Retail Sales Data Analysis exercise given in python textbook [Structured Thinking with Python](https://store.pothi.com/book/prithwis-mukerjee-structured-thinking-python/)


In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2025-08-22 07:04:50.854432+05:30
Python 3.12.11
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


#Install & Configure Postgres

For Connecting to a Postgres server on Amazon RDS, see [this notebook](https://github.com/Praxis-QR/RDWH/blob/main/PostgreSQL_01_Install_Config.ipynb)

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!sudo -u postgres psql -V
!psql -V
!service postgresql start



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


psql (PostgreSQL) 14.18 (Ubuntu 14.18-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


#Create Schema, Tables and Insert Data

In [3]:
# This has created an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:dbpraxis:upraxis:upass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:postgres:pass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:upraxis:upass
localhost:5432:dbpraxis:postgres:pass


In [4]:
# Get Retail Sales Data
!wget -q https://github.com/Praxis-QR/RDWH/raw/refs/heads/main/data/RetailSales51289.zip
!unzip RetailSales51289.zip
#!head RetailSales51289.csv

Archive:  RetailSales51289.zip
  inflating: RetailSales51289.csv    
order_date,ship_date,ship_mode,segment,state,country,market,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
2011-01-01,2011-01-06,Standard Class,Consumer,Constantine,Algeria,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408,2,0.0,106.14,35.46,Medium,2011
2011-01-01,2011-01-08,Standard Class,Consumer,New South Wales,Australia,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.036,9.72,Medium,2011
2011-01-01,2011-01-05,Second Class,Consumer,Budapest,Hungary,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66,4,0.0,29.64,8.17,High,2011
2011-01-01,2011-01-05,Second Class,Home Office,Stockholm,Sweden,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",45,3,0.5,-26.055,4.82,High,2011
2011-01-01,2011-01-08,Standard Class,Consumer,N

In [7]:
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists SalesData'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\
CREATE TABLE SalesData ( \
    order_date      DATE, \
    ship_date       DATE, \
    ship_mode       VARCHAR(50), \
    segment         VARCHAR(50), \
    state           VARCHAR(100), \
    country         VARCHAR(100), \
    market          VARCHAR(50), \
    region          VARCHAR(50), \
    product_id      VARCHAR(50), \
    category        VARCHAR(50), \
    sub_category    VARCHAR(50), \
    product_name    TEXT, \
    sales           NUMERIC(12,2), \
    quantity        INT, \
    discount        NUMERIC(5,2), \
    profit          NUMERIC(12,2), \
    shipping_cost   NUMERIC(12,2), \
    order_priority  VARCHAR(20), \
    year            INT \
); \
'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\dt public.*'
# --------------------------------------------------------------

NOTICE:  table "salesdata" does not exist, skipping
DROP TABLE
CREATE TABLE
                           List of relations
       Schema       |          Name           |    Type     |  Owner   
--------------------+-------------------------+-------------+----------
 information_schema | sql_features            | table       | postgres
 information_schema | sql_implementation_info | table       | postgres
 information_schema | sql_parts               | table       | postgres
 information_schema | sql_sizing              | table       | postgres
 pg_catalog         | pg_aggregate            | table       | postgres
 pg_catalog         | pg_am                   | table       | postgres
 pg_catalog         | pg_amop                 | table       | postgres
 pg_catalog         | pg_amproc               | table       | postgres
 pg_catalog         | pg_attrdef              | table       | postgres
 pg_catalog         | pg_attribute            | table       | postgres
 pg_catalog         | pg_

In [ ]:
# One way to handle UTF8 error
#
#!iconv -c -t utf8 eCommerce.csv > eCommerce.utf8.csv
#!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY ec.ecommerce FROM '/content/eCommerce.utf8.csv' DELIMITER ',' CSV HEADER;"

In [8]:
# Only superuser ('postgres') can use copy command, not even database owner ('upraxis')
# 'postgres' password has to be entered manually, password = 'pass'
#
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY salesdata FROM '/content/RetailSales51289.csv' DELIMITER ',' CSV HEADER ;"


COPY 51290


In [9]:
#!cat ItemsOrdered.csv
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'select * from SalesData limit 5'

 order_date | ship_date  |   ship_mode    |   segment   |      state      |  country  | market | region  |    product_id    |    category     | sub_category |        product_name         | sales  | quantity | discount | profit | shipping_cost | order_priority | year 
------------+------------+----------------+-------------+-----------------+-----------+--------+---------+------------------+-----------------+--------------+-----------------------------+--------+----------+----------+--------+---------------+----------------+------
 2011-01-01 | 2011-01-06 | Standard Class | Consumer    | Constantine     | Algeria   | Africa | Africa  | OFF-TEN-10000025 | Office Supplies | Storage      | Tenex Lockers, Blue         | 408.00 |        2 |     0.00 | 106.14 |         35.46 | Medium         | 2011
 2011-01-01 | 2011-01-08 | Standard Class | Consumer    | New South Wales | Australia | APAC   | Oceania | OFF-SU-10000618  | Office Supplies | Supplies     | Acme Trimmer, High Speed    | 120.00 |

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [11]:
#!pip install psycopg2
import psycopg2
import pandas as pd

In [12]:
# Local Colab server

conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.18 (Ubuntu 14.18-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [13]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

##Check Data

In [15]:
runSelect("select * from SalesData limit 10")

order_date,ship_date,ship_mode,segment,state,country,market,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
2011-01-01,2011-01-06,Standard Class,Consumer,Constantine,Algeria,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.00,2,0.00,106.14,35.46,Medium,2011
2011-01-01,2011-01-08,Standard Class,Consumer,New South Wales,Australia,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.00,3,0.10,36.04,9.72,Medium,2011
2011-01-01,2011-01-05,Second Class,Consumer,Budapest,Hungary,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.00,4,0.00,29.64,8.17,High,2011
2011-01-01,2011-01-05,Second Class,Home Office,Stockholm,Sweden,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",45.00,3,0.50,-26.06,4.82,High,2011
2011-01-01,2011-01-08,Standard Class,Consumer,New South Wales,Australia,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",114.00,5,0.10,37.77,4.70,Medium,2011
2011-01-01,2011-01-08,Standard Class,Consumer,New South Wales,Australia,APAC,Oceania,OFF-PA-10001968,Office Supplies,Paper,"Eaton Computer Printout Paper, 8.5 x 11",55.00,2,0.10,15.34,1.80,Medium,2011
2011-01-02,2011-01-06,Standard Class,Consumer,Ontario,Canada,Canada,Canada,TEC-OKI-10002750,Technology,Machines,"Okidata Inkjet, Wireless",314.00,1,0.00,3.12,24.10,Medium,2011
2011-01-03,2011-01-03,Same Day,Corporate,New South Wales,Australia,APAC,Oceania,OFF-AP-10000304,Office Supplies,Appliances,"Hoover Microwave, White",276.00,1,0.10,110.41,125.32,Critical,2011
2011-01-03,2011-01-09,Standard Class,Consumer,Auckland,New Zealand,APAC,Oceania,TEC-CO-10004182,Technology,Copiers,"Hewlett Wireless Fax, Laser",912.00,4,0.40,-319.46,107.10,Low,2011
2011-01-03,2011-01-07,Standard Class,Corporate,Ninawa,Iraq,EMEA,EMEA,FUR-NOV-10002791,Furniture,Chairs,"Novimex Swivel Stool, Set of Two",667.00,4,0.00,253.32,81.26,High,2011


#ChronoTYM <br>
In Indic phraseology, tantra is knowledge, a yantra is a device to explore this knowledge and a mantra is the code that unlocks both the tantra and its yantra. Chrono/TYM a sci-fi trilogy that recreates this framework where the first novel Chronotantra defines the contours of a tech driven, futuristic society, the second, Chronoyantra, describes a device used to explore the genesis of this society, while the third, Chronomantra finally reveals the code, the key, that unlocks this puzzle.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjdu_ajYwBRzk5iFYlcs59ZoRMMg6rhNm4JERKspvFJoveJsnfwRlzMICiC37UOja_Bd4sbfLA1pyyBbe0mj3tUfDBXoqCzNUDw3Wux7acoJBaLkWjAvJXfa7drRY2GQRAlFxEY9O7a7raY4RKy7cA3Uxgf5QsaZQ99C9GfXxFSex_rke8ROhAn41t8zOqK/s1640/tymBanner01_3.png)

In [ ]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2025-02-27 06:09:40.013195+05:30
